In [2]:
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image
import requests
from io import BytesIO

c:\aiproject2\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# TensorFlow MobileNetV2 모델 로드
model = tf.keras.applications.MobileNetV2(weights="imagenet")

In [4]:
def predict_image(image_url):
    try:
        # URL에서 이미지 가져오기
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content)).resize((224, 224))  # BytesIO 사용하여 이미지 열기

        # 이미지를 배열로 변환
        image_array = tf.keras.preprocessing.image.img_to_array(image)
        image_array = tf.expand_dims(image_array, axis=0)  # 배치 차원 추가
        image_array = tf.keras.applications.mobilenet_v2.preprocess_input(image_array)  # 전처리

        # 예측 수행
        predictions = model.predict(image_array)
        decoded_predictions = tf.keras.applications.mobilenet_v2.decode_predictions(predictions, top=3)[0]  # 상위 3개 예측 결과 반환

        # Gradio Label 컴포넌트에 맞게 결과 형식 변경
        # {label1: confidence1, label2: confidence2, ...} 형식으로 반환
        result = {label: float(prob) for (_, label, prob) in decoded_predictions}
        return result

    except Exception as e:
        return {"error": 1.0}  # 에러 발생 시 기본값 반환

In [5]:
# Gradio 인터페이스 생성
iface = gr.Interface(
    fn=predict_image,
    inputs=gr.Textbox(label="이미지 URL 입력"),
    outputs=gr.Label(num_top_classes=3, label="예측 결과"),
    title="음식 이미지 분류",
    description="이미지 URL을 입력하면 상위 3개의 예측 결과를 확률과 함께 표시합니다."
)

In [6]:
# 인터페이스 실행
iface.launch(server_port=7861, server_name="0.0.0.0", debug=True)

# 예시 이미지 URL : https://health.chosun.com/site/data/img_dir/2024/04/19/2024041901914_0.jpg

* Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [7]:
iface.close()

Closing server running on port: 7861


In [8]:
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image
import requests
from io import BytesIO
from langchain_community.chat_models import ChatOllama

In [9]:
# TensorFlow MobileNetV2 모델 로드
model = tf.keras.applications.MobileNetV2(weights="imagenet")

In [10]:
OLLAMA_SERVER = "http://localhost:11434"  # 로컬 서버 주소
MODEL_NAME = "gemma2"  # 사용하려는 Ollama 모델 이름

In [11]:
# Ollama를 사용해 음식 설명 생성
def get_food_description_with_langchain(food_name):
    """
    LangChain ChatOllama를 사용하여 음식 설명 생성
    """
    try:
        chat = ChatOllama(base_url=OLLAMA_SERVER, model=MODEL_NAME)
        prompt = f"{food_name}에 대해 말해줘."
        response = chat.predict(prompt)
        return response
    except Exception as e:
        return f"Failed to retrieve description: {e}"

In [12]:
# 이미지 예측 함수
def predict_image_with_description(image_url):
    """
    이미지 URL을 받아 음식 예측과 Ollama 설명을 반환
    """
    try:
        # URL에서 이미지 가져오기
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content)).resize((224, 224))  # BytesIO 사용하여 이미지 열기

        # 이미지를 배열로 변환
        image_array = tf.keras.preprocessing.image.img_to_array(image)
        image_array = tf.expand_dims(image_array, axis=0)  # 배치 차원 추가
        image_array = tf.keras.applications.mobilenet_v2.preprocess_input(image_array)  # 전처리

        # 예측 수행
        predictions = model.predict(image_array)
        decoded_predictions = tf.keras.applications.mobilenet_v2.decode_predictions(predictions, top=3)[0]  # 상위 3개 예측 결과 반환

        # 예측 결과 형식화
        result = {label: float(prob) for (_, label, prob) in decoded_predictions}

        # 가장 높은 확률의 예측값으로 Ollama 설명 생성
        top_food = decoded_predictions[0][1]  # 가장 확률이 높은 음식 이름
        description = get_food_description_with_langchain(top_food)

        return result, description  # 예측 결과와 Ollama 설명 반환

    except Exception as e:
        return {"error": 1.0}, f"Error: {e}"  # 에러 발생 시 기본값 반환

In [13]:
# Gradio 인터페이스 생성
iface = gr.Interface(
    fn=predict_image_with_description,
    inputs=gr.Textbox(label="이미지 URL 입력"),
    outputs=[
        gr.Label(num_top_classes=3, label="예측 결과"),  # 상위 3개 예측 결과
        gr.Textbox(label="음식 설명", interactive=False)  # Ollama로 생성한 설명 출력
    ],
    title="음식 이미지 분류 및 설명 생성기",
    description="이미지 URL을 입력하면 음식 분류 결과와 설명을 제공합니다."
)

In [14]:
# 인터페이스 실행
iface.launch(server_port=7861, server_name="0.0.0.0", debug=True)

* Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [15]:
iface.close()

Closing server running on port: 7861


In [16]:
import gradio as gr
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [17]:
# 워드 클라우드 생성함수수
def generate_wordcloud(file_obj):
    try:
        # 파일이 없는 경우 처리
        if file_obj is None:
            return None

        # Gradio의 파일 객체에서 파일 경로 가져오기
        file_path = file_obj.name

        # 파일 읽기
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()

        # 워드클라우드 생성
        wordcloud = WordCloud(
            font_path='malgun',  # 한글 폰트 설정 (맑은 고딕)
            background_color='white',
            width=800,
            height=600,
            max_words=200,
            max_font_size=100,
            min_font_size=10,
            random_state=42
        ).generate(text)

        # matplotlib 그래프 초기화
        plt.clf()

        # 워드클라우드 이미지를 플롯
        plt.figure(figsize=(10, 5))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.tight_layout()

        # 결과 이미지를 저장
        output_path = 'wordcloud.png'
        plt.savefig(output_path)
        plt.close()  # 메모리 누수 방지를 위해 figure 닫기
        return output_path

    except Exception as e:
        print(f"Error: {str(e)}")
        return None

In [18]:
# Gradio 인터페이스 생성
iface = gr.Interface(
    fn=generate_wordcloud,
    inputs=gr.File(label="Upload a .txt file"),
    outputs=gr.Image(type="filepath", label="Word Cloud")
)

In [19]:
iface.launch(server_port=7861, share=True, server_name="0.0.0.0")

* Running on local URL:  http://0.0.0.0:7861
* Running on public URL: https://4bceebed91eae26d5e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
iface.close()